In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import openpyxl as opxl
import flapjack
from flapjack import tables as tbc
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas.io.sql as psql
from itertools import islice
from datetime import timedelta
from datetime import time
import json

In [39]:
#df_json = pd.read_json('test.json', orient='records')
with open('test.json') as f:
    data_str = f.read()
data = json.loads(data_str)

In [44]:
meta = json.dumps(data)
df_meta = pd.read_json('test.json', orient='split')
#df_json = pd.read_json('test.json', orient='dict')

AttributeError: 'str' object has no attribute 'items'

In [24]:
file_name = 'Lector_metadata.xlsx'
experiment_name = os.path.basename(file_name).split('/')[-1].split('.')[0]
medidas = ['OD600:600', 'RFP-YFP:585/10,620/15', 'RFP-YFP:500/27,540/25', 'CFP:420/50,485/20']

wb = opxl.load_workbook(filename = file_name, data_only=True)
ws = wb['Data']
machine_name = ws['B'][8].value + str(ws['B'][9].value)

name_map = {'OD600:600':'OD', 'RFP-YFP:500/27,540/25':'YFP', 'CFP:420/50,485/20':'CFP',
'RFP-YFP:585/10,620/15':'RFP'}

/home/guillermo/anaconda3/lib/python3.6/site-packages/openpyxl/reader/worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [25]:
lista_rows = [(celda.value, celda.row, opxl.utils.column_index_from_string(celda.column)) 
              for celda in ws['A'] 
              if celda.value in medidas]
lista_rows

[('OD600:600', 47, 1),
 ('RFP-YFP:585/10,620/15', 148, 1),
 ('RFP-YFP:500/27,540/25', 249, 1),
 ('CFP:420/50,485/20', 350, 1)]

In [26]:
ws.delete_rows(1, 48)
data = ws.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, columns=cols)

In [27]:
df.columns = df.columns.map(lambda x: x.replace('T° OD600:600', 'T°'))

In [28]:
df_OD = pd.DataFrame(df.iloc[0:97])
df_OD['name'] = 'OD'

df_RFP = pd.DataFrame(df.iloc[101:198])
df_RFP['name'] = 'RFP'

df_YFP = pd.DataFrame(df.iloc[202:299])
df_YFP['name'] = 'YFP'

df_CFP = pd.DataFrame(df.iloc[303:400])
df_CFP['name'] = 'CFP'

In [207]:
# Código para arreglar los tiempos, si es que necesario
#t = df_OD['Time'].loc[0]
#t.minute
#t2 = time(t.minute-11)
#t2

In [29]:
# Provisorio
# Sacar strains
ws_strain = wb['Strains']
data = ws_strain.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df_strains = pd.DataFrame(data, index = idx, columns=cols)
df_strains

,1,2,3,4,5,6,7,8,9,10,11,12
A,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
B,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
C,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
D,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
E,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
F,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
G,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10
H,None,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10,Top10


In [187]:
for c, serie in df_strains.iteritems():
    for i in serie.index:
        print("Row/Col {}{}, valor {}".format(i, c, serie[i]))

Row/Col A1, valor None
Row/Col B1, valor None
Row/Col C1, valor None
Row/Col D1, valor None
Row/Col E1, valor None
Row/Col F1, valor None
Row/Col G1, valor None
Row/Col H1, valor None
Row/Col A2, valor Top10
Row/Col B2, valor Top10
Row/Col C2, valor Top10
Row/Col D2, valor Top10
Row/Col E2, valor Top10
Row/Col F2, valor Top10
Row/Col G2, valor Top10
Row/Col H2, valor Top10
Row/Col A3, valor Top10
Row/Col B3, valor Top10
Row/Col C3, valor Top10
Row/Col D3, valor Top10
Row/Col E3, valor Top10
Row/Col F3, valor Top10
Row/Col G3, valor Top10
Row/Col H3, valor Top10
Row/Col A4, valor Top10
Row/Col B4, valor Top10
Row/Col C4, valor Top10
Row/Col D4, valor Top10
Row/Col E4, valor Top10
Row/Col F4, valor Top10
Row/Col G4, valor Top10
Row/Col H4, valor Top10
Row/Col A5, valor Top10
Row/Col B5, valor Top10
Row/Col C5, valor Top10
Row/Col D5, valor Top10
Row/Col E5, valor Top10
Row/Col F5, valor Top10
Row/Col G5, valor Top10
Row/Col H5, valor Top10
Row/Col A6, valor Top10
Row/Col B6, valor Top10


In [164]:
# Provisorio
# Sacar media
ws_strain = wb['Media']
data = ws_strain.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df_media = pd.DataFrame(data, index = idx, columns=cols)
df_media

,1,2,3,4,5,6,7,8,9,10,11,12
A,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa
B,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa
C,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa
D,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa,M9-glucosa
E,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol
F,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol
G,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol
H,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol,M9-glicerol


In [188]:
for c, serie in df_media.iteritems():
    for i in serie.index:
        print("Row/Col {}{}, valor {}".format(i, c, serie[i]))

Row/Col A1, valor M9-glucosa
Row/Col B1, valor M9-glucosa
Row/Col C1, valor M9-glucosa
Row/Col D1, valor M9-glucosa
Row/Col E1, valor M9-glicerol
Row/Col F1, valor M9-glicerol
Row/Col G1, valor M9-glicerol
Row/Col H1, valor M9-glicerol
Row/Col A2, valor M9-glucosa
Row/Col B2, valor M9-glucosa
Row/Col C2, valor M9-glucosa
Row/Col D2, valor M9-glucosa
Row/Col E2, valor M9-glicerol
Row/Col F2, valor M9-glicerol
Row/Col G2, valor M9-glicerol
Row/Col H2, valor M9-glicerol
Row/Col A3, valor M9-glucosa
Row/Col B3, valor M9-glucosa
Row/Col C3, valor M9-glucosa
Row/Col D3, valor M9-glucosa
Row/Col E3, valor M9-glicerol
Row/Col F3, valor M9-glicerol
Row/Col G3, valor M9-glicerol
Row/Col H3, valor M9-glicerol
Row/Col A4, valor M9-glucosa
Row/Col B4, valor M9-glucosa
Row/Col C4, valor M9-glucosa
Row/Col D4, valor M9-glucosa
Row/Col E4, valor M9-glicerol
Row/Col F4, valor M9-glicerol
Row/Col G4, valor M9-glicerol
Row/Col H4, valor M9-glicerol
Row/Col A5, valor M9-glucosa
Row/Col B5, valor M9-glucos

In [165]:
# Provisorio
# Sacar DNA
ws_strain = wb['DNA']
data = ws_strain.values
cols = next(data)[1:]
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df_DNA = pd.DataFrame(data, index = idx, columns=cols)
df_DNA

,1,2,3,4,5,6,7,8,9,10,11,12
A,None,None,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP
B,None,None,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP
C,None,None,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP
D,None,None,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP
E,None,None,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP
F,None,None,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,std:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP,pLux76:RFP/std:YFP/std:CFP
G,None,None,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP
H,None,None,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pTet:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP,pLux76:RFP/pLacI:YFP/std:CFP


In [189]:
for c, serie in df_DNA.iteritems():
    for i in serie.index:
        print("Row/Col {}{}, valor {}".format(i, c, serie[i]))

Row/Col A1, valor None
Row/Col B1, valor None
Row/Col C1, valor None
Row/Col D1, valor None
Row/Col E1, valor None
Row/Col F1, valor None
Row/Col G1, valor None
Row/Col H1, valor None
Row/Col A2, valor None
Row/Col B2, valor None
Row/Col C2, valor None
Row/Col D2, valor None
Row/Col E2, valor None
Row/Col F2, valor None
Row/Col G2, valor None
Row/Col H2, valor None
Row/Col A3, valor std:RFP/std:YFP/std:CFP
Row/Col B3, valor std:RFP/std:YFP/std:CFP
Row/Col C3, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col D3, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col E3, valor std:RFP/std:YFP/std:CFP
Row/Col F3, valor std:RFP/std:YFP/std:CFP
Row/Col G3, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col H3, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col A4, valor std:RFP/std:YFP/std:CFP
Row/Col B4, valor std:RFP/std:YFP/std:CFP
Row/Col C4, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col D4, valor pLux76:RFP/pTet:YFP/std:CFP
Row/Col E4, valor std:RFP/std:YFP/std:CFP
Row/Col F4, valor std:RFP/std:YFP/std:CFP
Row/Col G4, valor pL